<h2>Building Scribr - An Autocomplete Tool for Scribes

<p>First, I scraped a glossary of medical terms which was a good source of data for a few reasons:

<p>1) <b>Consistency</b> - there's minimal variance page to page (fonts, bolding, etc), which made scraping much easier

<p>2) <b>Related terms</b> - for some definitions, they're followed by a "See [condition]" which allows for a wider scope of  recommendations that aren't just based on the same prefixes

<h3>Scraping the PDF

In [10]:
from docx import *
import re

In [11]:
# scrubbing the invalid text
def valid_text(text):
    if 'INDEX' in text or 'Page' in text or text == ' ':
        return False
    else:
        return True

# import document
document = Document('Gloss_Med_Terms.docx')
terms = {}
fulllen_text = ''

# get key terms --> (key terms are bolded)
for para in document.paragraphs:
    for run in para.runs[0:]:
        fulllen_text += run.text
        if run.bold and valid_text(run.text):
            text = run.text
            terms[text.strip()] = 1

# get definitions & key terms again just to double check
items = []
splt = fulllen_text.split("--")
for spl in splt[1:]:
    # remove bad terms
    cleaned_text = re.sub('Page \d+', ' ', spl)
    cleaned_text = re.sub('INDEX', ' ', cleaned_text)
    
    # split term and definition
    period_location = cleaned_text.rfind('.') + 1
    definition = cleaned_text[0:period_location].strip(" ")
    term = cleaned_text[period_location:].strip(" ")

    # add both to same list
    items.append(definition)
    items.append(term)

In [12]:
# quick sanity check
len(items)

2016

In [13]:
# hardcoded first definition 
terms['ACTH (Adrenocorticotropic hormone)'] = items[0]

# adding definitions
j = 1
while j < len(items) - 1:
    term = items[j]
    defn = items[j + 1]
    terms[term] = defn
    j += 2
    
# removing bad terms
bad_terms = []
for key in terms:
    if terms[key] == 1:
        bad_terms.append(key)
        
for term in bad_terms:
    del terms[term]

<h3>Creating a Glossary of Medical Terms (and their Relatives)

<p>Here, I'm just building a simple hashmap of 'term' to 'MedicalTerm', which is an object that stores the term itself, its definition, related terms, and a score. 

The term is put in all lowercase such that when the user enters queries on the frontend, case sensitivty does not find incorrect matches.

<p>*** Note: currently, the score is a random integer between 1 and 10 and as users click on a suggested result, the score would be adjusted accordingly

In [14]:
import random

# object to store medical terms
class MedicalTerm:
    
    def __init__(self, defn, term, related_terms=[]):
        self.definition = defn
        self.term = term
        self.related_terms = related_terms
        self.score = random.randint(0, 10)

In [15]:
# function to parse related terms from medical definitions
complete_related_terms = []

def get_related_terms(defn):
    # check if related terms exist
    start_loc = defn.find('See')
    if start_loc != -1:
        end_loc = defn.find('.', start_loc)
        defn = defn[start_loc:end_loc + 1]
        defn = defn[defn.find('See'):]
        defn = defn[defn.find(' '):]
        related_terms = defn.strip(' ').split(';')
        for i in range(0, len(related_terms)):
            related_terms[i] = related_terms[i].strip('. ').lower()
        
        return related_terms
    
# create mapping
medical_dictionary = {}
for term in terms:
    defn = terms[term]
    word = term.lower()
    medical_dictionary[word] = MedicalTerm(defn, word, get_related_terms(defn))

<h3>Building A Trie For Efficient Term Suggestions

<p>As we landed on in my interview, the ideal data structure for a
autocomplete suggestion is a trie. Below, you'll find the implementation
for it as well as some queries just to test it.

In [69]:
class Node:
    '''
    Each node stores a value, a MedicalTerm object (as defined above), 
    children, and a `completed` boolean to denote that a term is complete.
    '''
    def __init__(self, val='', m_term=None):
        self.val = val
        self.medical_term = m_term
        self.children = {}
        self.completed = False
        
class AutocompleteTrie:
    
    def __init__(self):
        '''
        Initializes an empty node as the root with an 
        empty dictionary as children.
        '''
        self.root = Node()
        
    
    def insert_word(self, m_term):
        '''
        Inserts medical term into the trie. Creates
        new children nodes correspondingly.
        '''
        val = m_term.term
        curr_node = self.root
        
        for i in range(0, len(val)):
            char = val[i]
            
            # create new nodes if children don't exist
            if char not in curr_node.children:
                stubbed_suggestion = val[0: i + 1]
                new_node = Node(stubbed_suggestion)
                curr_node.children[char] = new_node
                curr_node = new_node
            else:
                curr_node = curr_node.children[char]
        
        curr_node.medical_term = m_term
        curr_node.completed = True
    
    
    def find_word(self, val):
        '''
        Attempts to find a word in the trie. Returns False
        if the word is not found, otherwise returns the term, 
        definition, and related words in a Tuple:
            (term, definition, related_words)
        '''
        curr_node = self.root
        
        for k in val:
            if k not in curr_node.children:
                return False
            else:
                curr_node = curr_node.children[k]
                
        if curr_node.completed:
            return (curr_node.val, curr_node.medical_term.definition, curr_node.medical_term.related_terms)
        else:
            return False

    
    def find_suggestions(self, val):
        '''
        Returns a list of words that match a given prefix as well as words
        that could possibly be related to a given prefix.
        '''
        curr_node = self.root
        
        for k in val:
            if k not in curr_node.children:
                return list()
            
            curr_node = curr_node.children[k]
            
        suggestions = []
        self.find_suggestions_helper(curr_node, suggestions)
        
        return suggestions
    
    
    def find_suggestions_helper(self, node, suggestions):
        '''
        Recursive helper function for the above.
        '''
        if node.completed:
            suggestions.append(node.val)
            # add related terms to suggestions if they exist
            if node.medical_term.related_terms:
                suggestions.extend(node.medical_term.related_terms)
        
        for c in node.children:
            self.find_suggestions_helper(node.children[c], suggestions)


<h3>3, 2, 1...Takeoff

In [70]:
# creating the trie & stuffing it with our medical terms
autocomplete_trie = AutocompleteTrie()
for term in medical_dictionary:
    autocomplete_trie.insert_word(medical_dictionary[term])

<p>Let's see what happens when we search for a word...like 'abscess'

In [71]:
autocomplete_trie.find_word('abscess')

('abscess',
 'Swollen, inflamed, tender area of infection filled with pus.',
 None)

<p>Neat, what about when we try and find an <b>exact</b> match for 'ab', which isn't in our medical glossary?

In [72]:
autocomplete_trie.find_word('ab')

False

<p>What about when we try to find suggestions for medical terms that might start with 'ab'?

In [75]:
suggestions = autocomplete_trie.find_suggestions('ab')
suggestions

['abell-kendall modification', 'abruptio placenta', 'abscess']

<p>Hmm...that's cool, but let's examine this hypothetical scenario for just a second:
A member comes in with 'Ahaptoglobulinemia', which the scribe hears that through the intercom, but only gets the first few letters ('aha'). 
    
Here's what a potential search would look like:

In [76]:
autocomplete_trie.find_suggestions('aha')

['ahaptoglobulinemia', 'anemia, hemolytic', 'infectious mononucleosis']

Lo and behold, we'll see that specific condition as well as potentially related conditions that don't necessarily start with 'aha'. 

<h2>Analysis & Optimizations

Let's analyze the time complexity for each of the three main operations performed on this trie:
- Insert Word - O(n), where n is the number of characters in our term
- Find Word (Exact Match) - O(n), where n is the number of characters 
- Find Suggestions - O(n) + O(m) + O(q log q) (for sorting), where n is the number of characters in our query & m and q are the number of completions

Based on this, it's clear that 'find suggestions' function is the bottleneck in our application's performance. In order to eliminate this bottleneck, we can store completions in each Node in our trie. In other words, if we were looking for a term that started with 'ab', we would travel down the 'a' node to the 'b' node, which would store all possible completions for words beginning with 'ab'. Consequently, we wouldn't have to recursively travel down each child of 'ab' as we are in the status quo.

Our new trie is implemented below. Changes are marked with the ever-so subtle comment: "# NEW NEW NEW NEW NEW NEW"

In [77]:
import bisect

class Node:
    '''
    Each node stores a value, a MedicalTerm object (as defined above), 
    children, and a `completed` boolean to denote that a term is complete.
    '''
    def __init__(self, val='', m_term=None):
        self.val = val
        self.medical_term = m_term
        self.children = {}
        self.completions = []
        self.completed = False
        
class AutocompleteTrie:
    
    def __init__(self):
        '''
        Initializes an empty node as the root with an 
        empty dictionary as children.
        '''
        self.root = Node()
        
    
    def insert_word(self, m_term):
        '''
        Inserts medical term into the trie. Creates
        new children nodes correspondingly.
        '''
        val = m_term.term
        defn = m_term.definition
        curr_node = self.root
        
        # NEW NEW NEW NEW
        
        # add completions & their definitions if they exist
        related_terms_exist = False
        if m_term.related_terms:
            related_terms_exist = True
            
            completed_terms_and_definitions = []
            for term in m_term.related_terms:
                if term in medical_dictionary:
                    completed_terms_and_definitions.append((term, medical_dictionary[term].definition, medical_dictionary[term].score))
            
        # NEW NEW NEW NEW
        
        for i in range(0, len(val)):
            char = val[i]
            
            # create new nodes if children don't exist
            if char not in curr_node.children:
                stubbed_suggestion = val[0: i + 1]
                new_node = Node(stubbed_suggestion)
                
                # NEW NEW NEW NEW
                # append completions
                new_node.completions.append((val, defn, m_term.score))
                if related_terms_exist:
                    new_node.completions.extend(completed_terms_and_definitions)
                # NEW NEW NEW NEW 
                
                curr_node.children[char] = new_node
                curr_node = new_node
            else:
                curr_node = curr_node.children[char]
                
                # NEW NEW NEW NEW
                # append completions
                curr_node.completions.append((val, defn, m_term.score))
                if related_terms_exist:
                    curr_node.completions.extend(completed_terms_and_definitions)
                # NEW NEW NEW NEW
        
        curr_node.medical_term = m_term
        curr_node.completed = True
    
    
    def find_word(self, val):
        '''
        Attempts to find a word in the trie. Returns False
        if the word is not found, otherwise returns the term, 
        definition, and related words in a Tuple:
            (term, definition, related_words)
        '''
        curr_node = self.root
        
        for k in val:
            if k not in curr_node.children:
                return False
            else:
                curr_node = curr_node.children[k]
                
        if curr_node.completed:
            return curr_node
        else:
            return False

    
    def find_suggestions(self, val):
        '''
        Returns a list of words that match a given prefix as well as words
        that could possibly be related to a given prefix.
        '''
        curr_node = self.root
        
        for k in val:
            if k not in curr_node.children:
                return list()
            
            curr_node = curr_node.children[k]
            
        return sorted(curr_node.completions, key=lambda x: x[2], reverse=True)

<h3>Testing Our New Implementation

In [78]:
# creating the trie & stuffing it with our medical terms
autocomplete_trie = AutocompleteTrie()
for term in medical_dictionary:
    autocomplete_trie.insert_word(medical_dictionary[term])

In [79]:
dummy = autocomplete_trie.find_word('alkalosis, respiratory')
dummy.completions

[('alkalosis, respiratory',
  'abdominal pain, diarrhea, appetite loss and brown skin. Abnormal condition when body fluids are more alkaline than normal. Caused by conditions that decrease the level of carbon dioxide in the blood, such as breathing too rapidly or congestive heart failure. See Congestive heart failure.',
  3),
 ('congestive heart failure',
  'Complication of many serious diseases in which the heart loses its full pumping capacity. Blood backs up into other organs, especially the lungs and liver.',
  2)]

In [81]:
suggestions = autocomplete_trie.find_suggestions('aha')
for suggestion in suggestions:
    print("Term: ", suggestion[0])
    print("Definition: ", suggestion[1])
    print("Score: ", suggestion[2], "\n")

Term:  anemia, hemolytic
Definition:  Anemia due to the premature destruction of mature red blood cells. Bone marrow cannot produce red blood cells fast enough to compensate for those being destroyed.
Score:  4 

Term:  ahaptoglobulinemia
Definition:  Without haptoglobin in the blood. Condition is often seen with hemolytic anemia, severe liver disease and infectious mononucleosis. See Anemia, hemolytic; infectious mononucleosis.
Score:  2 

Term:  infectious mononucleosis
Definition:  Infectious viral disease that affects the liver, respiratory system and lymphatic system.
Score:  0 



<h3>Final Notes

The solution described above still isn't the ideal solution. In an ideal solution, we'd like O(1) reads & writes (assuming the terms are presorted). We can use the optimization above and take it a step further by hashing every single suggestion to a possible list of completions, where we would ultimately be trading space for performance. A NoSQL database implementation (i.e Redis)  would be ideal for something like this. But for the purposes of this, simply using a NoSQL database instead of implementing my own data structure from scratch would qualify as cheating.

An example of the aforementioned optimization:<br>
a --> addition(score:20), app(score: 15), apple(score: 10), application(score:5)<br>
ad --> addition(score:20)<br>
app --> apple(score:10), application(score:5)<br>
appl --> apple(score:10), application(score:5)<br>
apple --> apple(score:10)<br>
appli --> application(score:5)<br>
  